In [1]:
import sys

import _pickle as cPickle
import numpy as np
import csv
#from models import *
from models_small_pretrained_train import *
from feature_extraction import train_test_features, dumpFeatures
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#dumpFeatures(False, False, True, None, 'features_pretrained_small.dat')
[X_train, y, X_test, max_features, W] = cPickle.load(open("features_pretrained_small.dat", "rb"))
#model = build_model2(max_features+1, W.shape[1], X_train.shape[1], embeddings=W, trainable=True)




#keras_model = KerasClassifier(build_fn=build_model, verbose=0)
#keras_model = KerasClassifier(build_fn=model, verbose=0)
keras_model = KerasClassifier(build_fn=build_model2, verbose=0)
param_grid = {
    
    #'regularization': [0.01, 0.1, 0.2],
    #'weight_constraint': [1., 2., 3.],
    #'dropout_prob': [0.2, 0.4, 0.5, 0.6, 0.7],
    #'epochs': [2, 3],
    #'batch_size': [64, 128, 160]
    'epochs': [1],
    'batch_size': [128],
    'kernel': [2],
}


legal_params:  {'callbacks', 'batch_size', 'verbose', 'class_weight', 'shuffle', 'input_dim', 'initial_epoch', 'validation_data', 'self', 'y', 'epochs', 'validation_split', 'output_dim', 'x', 'sample_weight', 'input_length'}


In [3]:
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, verbose=2, cv=2)
grid_result = grid.fit(X_train, y)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
legal_params:  {'callbacks', 'batch_size', 'verbose', 'class_weight', 'shuffle', 'input_dim', 'initial_epoch', 'validation_data', 'self', 'y', 'epochs', 'validation_split', 'output_dim', 'x', 'sample_weight', 'input_length'}
legal_params:  {'callbacks', 'batch_size', 'verbose', 'class_weight', 'shuffle', 'input_dim', 'initial_epoch', 'validation_data', 'self', 'y', 'epochs', 'validation_split', 'output_dim', 'x', 'sample_weight', 'input_length'}
[CV] batch_size=128, epochs=1, kernel=2 ..............................
legal_params:  {'callbacks', 'batch_size', 'verbose', 'class_weight', 'shuffle', 'input_dim', 'initial_epoch', 'validation_data', 'self', 'y', 'epochs', 'validation_split', 'output_dim', 'x', 'sample_weight', 'input_length'}


ValueError: kernel is not a legal parameter

In [4]:
model = build_model2()
model.fit(X_train, y, validation_split=0.1, nb_epoch=2, batch_size=128, verbose=1)

model 2


/usr/local/lib/python3.6/dist-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 180000 samples, validate on 20000 samples
Epoch 1/2
180000/180000 [==============================] - 291s - loss: 0.3905 - acc: 0.8145 - val_loss: 0.3699 - val_acc: 0.8294
Epoch 2/2
180000/180000 [==============================] - 299s - loss: 0.3140 - acc: 0.8575 - val_loss: 0.3647 - val_acc: 0.8347


Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] batch_size=128, epochs=1, kernel=2 ..............................


ValueError: kernel is not a legal parameter

In [35]:
# grid_result.best_estimator_.model.summary()
grid_result.best_params_

grid_result.best_estimator_.model.save("testdump")

In [36]:
from keras.models import load_model

new_model = load_model('testdump')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 30, 200)           19267800  
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 26, 128)           128128    
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 5, 128)            0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 1, 128)            82048     
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 19,478,105
Trainable params: 19,478,105
Non-trainable params: 0
________________________________________________________________

In [37]:
grid_result.grid_scores_

cPickle.dump(grid_result.best_params_, open('features_pretrained_small/testdump', 'wb'))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [24]:
cPickle.load(open("testdump", "rb"))

{'batch_size': 128, 'epochs': 1}

In [ ]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


joblib.dump(grid.best_estimator_, 'features_pretrained_small_MODEL.pkl')